## Pre-processing

In [ ]:
import os
import cv2
import numpy as np
from string import ascii_uppercase

In [ ]:
train_dir='./train'
train_dir_img=train_dir+'/{}'

train=[train_dir_img.format(i) for i in os.listdir(train_dir)]
train=[f for f in train if not f.endswith('DS_Store')]

val_dir='./val'
val_dir_img=val_dir+'/{}'

val=[val_dir_img.format(i) for i in os.listdir(val_dir)]
val=[f for f in val if not f.endswith('DS_Store')]

test_dir='./test'
test_dir_img=test_dir+'/{}'

test=[test_dir_img.format(i) for i in os.listdir(test_dir)]
test=[f for f in test if not f.endswith('DS_Store')]

In [ ]:
characters=[c for c in ascii_uppercase if c not in "JZ"]
nrows=100
ncolumns=100

In [ ]:
def read_and_process(list_of_images): 
    lower = 100
    upper = 125
    
    X=[]
    y=[]
    for image in list_of_images:
        img = cv2.imread(image,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (nrows, ncolumns), interpolation=cv2.INTER_CUBIC)
        blur_img = cv2.GaussianBlur(img,(3,3),0)      
        edge = cv2.Canny(blur_img,lower,upper)
        
        X.append(edge)   
        for c in characters:
            if c in image:
                y.append(c)
    
    return X,y

In [ ]:
train_X, train_y = read_and_process(train)
val_X, val_y = read_and_process(val)
test_X, test_y = read_and_process(test)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

OHE=OneHotEncoder(sparse=False)

train_y = OHE.fit_tansform(train_y)
val_y = OHE.fit_tansform(val_y)
test_y = OHE.fit_tansform(test_y)

In [ ]:
train_X = np.reshape(np.array(train_X), (len(train_X),200,200,1))
val_X = np.reshape(np.array(val_X), (len(val_X),200,200,1))
test_X = np.reshape(np.array(test_X), (len(test_X),200,200,1))

train_y = np.array(train_y)
val_y = np.array(val_y)
test_y = np.array(test_y)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen =  ImageDataGenerator(rescale=1.0/255, 
                                    rotation_range=15,
                                    zoom_range=0.2)

val_datagen =  ImageDataGenerator(rescale=1.0/255)

test_datagen =  ImageDataGenerator(rescale=1.0/255)

In [ ]:
batch_size=50

train_gen = train_datagen.flow(train_X,
                               train_y,
                               batch_size=batch_size,
                               shuffle=True,
                               seed=0)
val_gen = val_datagen.flow(val_X,
                           val_y,
                           batch_size=batch_size,
                           shuffle=True,
                           seed=0)

test_gen = test_datagen.flow(test_X,
                             test_y,
                             batch_size=batch_size,
                             shuffle=True,
                             seed=0)

## Model

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

In [ ]:
def CaffeNet(input_shape=(100,100,1),nclass=24)
    #Instantiate an empty model
    model = Sequential()

    # 1st Convolutional Layer
    model.add(Conv2D(filters=96, input_shape=input_shape, kernel_size=(11,11), strides=(4,4), padding='same'))
    model.add(Activation('relu'))
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))
    model.add(BatchNormalization())

    # 2nd Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same'))
    model.add(Activation('relu'))
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))
    model.add(BatchNormalization())

    # 3rd Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
    model.add(Activation('relu'))

    # 4th Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
    model.add(Activation('relu'))

    # 5th Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
    model.add(Activation('relu'))
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))

    # Passing it to a Fully Connected layer
    model.add(Flatten())
    # 1st Fully Connected Layer
    model.add(Dense(4096))
    model.add(Activation('relu'))
    # Add Dropout to prevent overfitting
    model.add(Dropout(0.5))

    # 2nd Fully Connected Layer
    model.add(Dense(4096))
    model.add(Activation('relu'))
    # Add Dropout
    model.add(Dropout(0.5))

    # Output Layer
    model.add(Dense(nclass))
    model.add(Activation('softmax'))

    model.summary()

    # Compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
def VGG19(input_shape=(100,100,1),nclass=24)    
    model = Sequential()

    # Block 1
    model.add(Conv2D(filters=64,
                     input_shape=input_shape,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))

    model.add(Conv2D(filters=64,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))

    model.add(MaxPooling2D(pool_size=(2, 2),
                           strides=(2, 2)))

    # Block 2
    model.add(Conv2D(filters=128,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))

    model.add(Conv2D(filters=128,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))

    model.add(MaxPooling2D(pool_size=(2, 2),
                           strides=(2, 2)))

    # Block 3
    model.add(Conv2D(filters=256,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))

    model.add(Conv2D(filters=256,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))

    model.add(Conv2D(filters=256,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))

    model.add(Conv2D(filters=256,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))

    model.add(MaxPooling2D(pool_size=(2, 2),
                           strides=(2, 2)))

    # Block 4
    model.add(Conv2D(filters=512,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))

    model.add(Conv2D(filters=512,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))

    model.add(Conv2D(filters=512,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))

    model.add(Conv2D(filters=512,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))

    model.add(MaxPooling2D(pool_size=(2, 2),
                           strides=(2, 2)))

    # Block 5
    model.add(Conv2D(filters=512,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))

    model.add(Conv2D(filters=512,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))

    model.add(Conv2D(filters=512,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))

    model.add(Conv2D(filters=512,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))

    model.add(MaxPooling2D(pool_size=(2, 2),
                           strides=(2, 2)))

    # Passing it to a Fully Connected layer
    model.add(Flatten())

    # 1st Fully Connected Layer
    model.add(Dense(4096, activation='relu'))

    # 2nd Fully Connected Layer
    model.add(Dense(4096, activation='relu'))

    # Output layer
    model.add(Dense(nclass, activation='softmax'))

    model.summary()

    # Compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
def from_paper(input_shape=(100,100,1),nclass=24):
    model = Sequential()
    
    model.add(Conv2D(filters=16,
                     input_shape=input_shape,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(filters=32,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))
    
    model.add(MaxPooling2D(pool_size=(5, 5)))
    
    model.add(Conv2D(filters=64,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))
    
    model.add(MaxPooling2D(pool_size=(5, 5)))
    
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    
    model.add(Dense(1024, activation='relu'))
    
    model.add(Dense(nclass, activation='softmax'))
    
    model.summary()
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    
    return model

## Training

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
filepath = 'weight.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', 
                        verbose=1, save_best_only=True, mode='max')

early = EarlyStopping(monitor='val_loss', 
                      mode='min', 
                      patience=4, restore_best_weights=True)

callbacks_list = [checkpoint, early]

In [ ]:
input_shape=(nrows,ncolumns,1)
nclass=24
model = from_paper(input_shape,nclass)
#model = VGG19(input_shape,nclass)
#model = CaffeNet(input_shape,nclass)
history = model.fit_generator(train_gen,
                              steps_per_epoch=nclass*1000/batch_size,          
                              validation_data=val_gen,
                              validation_steps=batch_size, 
                              epochs=50, verbose=1,
                              callbacks=callbacks_list)

In [ ]:
model.save_model('model.h5')

## Visualization

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()

plt.savefig('accuracy_loss.png')